# Choosing Comparison Levels

When building a Splink model, one of the most important aspects is defining the [`Comparisons`](../comparison.md) and [`Comparison Levels`](../comparison_level.md) that the model will train on. Each `Comparison Level` within a `Comparison` should contain a different amount of evidence that two records are a match, which the model can assign a Match Weight to. 


## How Comparison Levels work



## How records get assigned to Comparison Levels

When assigning a record pair to a `Comparison Level`, Splink runs sequentially through each level and assigns the record pair to the first level that they satisfy the condition of. As a result, when defining a `Comparison`, care must be taken on order of the `Comparison Level`s.

For example, if we construct two simple comparisons for a `name` column, both with an exact match and a fuzzy match ([levenshtein](./comparators.md#levenshtein-distance)) level:

In [1]:
import splink.duckdb.comparison_level_library as cll

comparison_correct_order = {
    "output_column_name": "name",
    "comparison_levels": [
        cll.null_level("name"),
        cll.exact_match_level("name"),
        cll.levenshtein_level("name", 2),
        cll.else_level(),
    ],
}

comparison_incorrect_order = {
    "output_column_name": "name",
    "comparison_levels": [
        cll.null_level("name"),
        cll.levenshtein_level("name", 2),
        cll.exact_match_level("name"),
        cll.else_level(),
    ],
}

Now, consider a list of names to consider and we can use the `get_comparison_levels` function from the comparison_hepers library to see how each combination of these names are assigned to a `Comparison Level` for our two `Comparison`s

In [2]:
import splink.comparison_helpers as ch

names = ["Julia", "Julia", "Julie", "Rachel"]

ch.get_comparison_levels(names, comparison_correct_order)

,,name
comparison_level,gamma,
Exact match,2,"[[Julia, Julia]]"
Levenshtein <= 2,1,"[[Julia, Julie], [Julia, Julie]]"
All other comparisons,0,"[[Julia, Rachel], [Julia, Rachel], [Julie, Rac..."
Null,-1,[]


In [3]:
ch.get_comparison_levels(names, comparison_incorrect_order)

,,name
comparison_level,gamma,
Levenshtein <= 2,2,"[[Julia, Julia], [Julia, Julie], [Julia, Julie]]"
Exact match,1,[]
All other comparisons,0,"[[Julia, Rachel], [Julia, Rachel], [Julie, Rac..."
Null,-1,[]


## 

## Debugging Comparison Levels